# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, r2_score
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

display(data.head(10))

data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
data.isna().mean()

RowNumber          0.0000
CustomerId         0.0000
Surname            0.0000
CreditScore        0.0000
Geography          0.0000
Gender             0.0000
Age                0.0000
Tenure             0.0909
Balance            0.0000
NumOfProducts      0.0000
HasCrCard          0.0000
IsActiveMember     0.0000
EstimatedSalary    0.0000
Exited             0.0000
dtype: float64

In [4]:
data.duplicated().sum()

0

In [5]:
data.nunique().sort_values()

Gender                 2
HasCrCard              2
IsActiveMember         2
Exited                 2
Geography              3
NumOfProducts          4
Tenure                11
Age                   70
CreditScore          460
Surname             2932
Balance             6382
EstimatedSalary     9999
RowNumber          10000
CustomerId         10000
dtype: int64

Пропуски в колонке Tenure. Они составляют 9%.
Дубликатов и повтораяющихся значений нет

In [6]:
#Удалим столбцы CustomerId, RowNumber, Surname - так как для обучения модели они не пригодятся
data = data.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1)

In [7]:
#Посмотрим, как распределены пропуски в данных между клиентами из разных стран по половому признаку

t_nans = data[data['Tenure'].isna()].groupby(['Geography', 'Gender'])['Exited'].count().reset_index()
t_all = data.groupby(['Geography', 'Gender'])['Exited'].count().reset_index()
t = pd.merge(t_nans, t_all, on=['Geography', 'Gender'])
t['ratio'] = t['Exited_x'] / t['Exited_y']
display(t)

,Geography,Gender,Exited_x,Exited_y,ratio
0,France,Female,217,2261,0.095975
1,France,Male,247,2753,0.089720
2,Germany,Female,108,1193,0.090528
3,Germany,Male,108,1316,0.082067
4,Spain,Female,101,1089,0.092746
5,Spain,Male,128,1388,0.092219


Пропуски распределены равномерно (примерно в одинаковом соотношении) по группам.

In [8]:
data['Exited'].mean()

0.2037

In [9]:
data[data['Tenure'].isna()]['Exited'].mean()

0.20132013201320131

Баланс классов у пропусков примерно такой же как в датасете.

In [10]:
#Определим категориальные и численные переменные

cat_columns = ['Gender',
               'HasCrCard',
               'IsActiveMember',
               'Geography',
               'NumOfProducts',
               'Tenure'
]

num_columns = ['Age',
               'CreditScore',
               'Balance',
               'EstimatedSalary',
]


data.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


- В столбце Tenure есть пропущенные данные: данные могли отсутствовать изначально или быть утеряны из-за сбоев в системе при выгрузке базы.

- Попробуем построить модель, предсказывающую данный параметр на основе имеющихся данных.

In [11]:
# сохраним данные с пропущенными значениями 'Tenure' в отдельной переменной
no_tenure_df = data.loc[data['Tenure'].isna()]
features = data.loc[~data['Tenure'].isna()]

# основной целевой признак 'Exited' не будет участвовать в данной модели
features = features.drop('Exited', axis=1)

# в данной модели признак 'Tenure' выступает в качестве целевого
y = features['Tenure']
X = features.drop('Tenure', axis=1)
X = features[['EstimatedSalary', 'Age', 'CreditScore']]

# разделим данные на обучающую и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=123)

In [12]:
#Классификацию попробуем провести с помощью модели "Случайный Лес". 
#Так как признак 'Tenure' распределён равномерно между 11 значениями, попробуем оценивать качество модели с помощью accuracy.
#в параметрах best_params и best_accuracy будем хранить наилучшие параметры для модели и наивысшую точность соответственно
best_params = []
best_accuracy = 0
# проходим в циклах по всем выбранным гиперпараметрам
for n_estimators in (1,10):
    for max_depth in range(2,10):
        for min_samples_split in range(2,10):
            for min_samples_leaf in range(1,8):
                for criterion in ['gini', 'entropy']:
                    # инициация модели с текущими гиперпараметрами
                    model_forest = RandomForestClassifier(random_state=123,
                                                          n_estimators=n_estimators,
                                                          max_depth=max_depth,
                                                          min_samples_split=min_samples_split,
                                                          min_samples_leaf=min_samples_leaf,
                                                          criterion=criterion)
                    # обучение модели на тренировочной выборке
                    model_forest.fit(X_train, y_train)
                    # поиск предсказаний модели на валидационной выбоке
                    predictions = model_forest.predict(X_valid)
                    # вычисление точности модели методом accuracy_score
                    accuracy = accuracy_score(y_valid, predictions)
                    # если текущее значение точности выше предыдущего лучшего значения, 
                    # сохраняем параметры модели и текущую точность
                    if accuracy > best_accuracy:
                        best_params = [n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion]
                        best_accuracy = accuracy

In [13]:
# вывод наилучшей точности и параметров модели.
print('''Наилучшая точность: {}
Параметры модели:
   n_estimators = {}
   max_depth = {}
   min_samples_split = {}
   min_samples_leaf = {}
   criterion = {} '''.format(round(best_accuracy, 2),
                             best_params[0],
                             best_params[1],
                             best_params[2],
                             best_params[3],
                             best_params[4]
                            ))

Наилучшая точность: 0.12
Параметры модели:
   n_estimators = 10
   max_depth = 5
   min_samples_split = 2
   min_samples_leaf = 2
   criterion = entropy 


Accuracy модели 12% слишком мал (случайный выбор между 11 классами даст около 9%). Другие модели классификации не улучшили качество классификации.

Регрессионные модели (представление Tenure количественным признаком) не дали положительного результата (r2 близок к 0 для LinearRegression и RandomForestRegressor).

В данном случае представим признак Tenure категориальным, преобразуем его методом One-Hot Encoding, пропущенные значения будут отдельной категорией.

In [14]:
data.loc[data['Tenure'].isna(), 'Tenure'] = -1
data['Tenure'] = data['Tenure'].astype('object')
data = pd.get_dummies(data, drop_first=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Balance            10000 non-null  float64
 3   NumOfProducts      10000 non-null  int64  
 4   HasCrCard          10000 non-null  int64  
 5   IsActiveMember     10000 non-null  int64  
 6   EstimatedSalary    10000 non-null  float64
 7   Exited             10000 non-null  int64  
 8   Geography_Germany  10000 non-null  uint8  
 9   Geography_Spain    10000 non-null  uint8  
 10  Gender_Male        10000 non-null  uint8  
 11  Tenure_0.0         10000 non-null  uint8  
 12  Tenure_1.0         10000 non-null  uint8  
 13  Tenure_2.0         10000 non-null  uint8  
 14  Tenure_3.0         10000 non-null  uint8  
 15  Tenure_4.0         10000 non-null  uint8  
 16  Tenure_5.0         1000

In [15]:
display(data.head(10))

,CreditScore,Age,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,...,Tenure_1.0,Tenure_2.0,Tenure_3.0,Tenure_4.0,Tenure_5.0,Tenure_6.0,Tenure_7.0,Tenure_8.0,Tenure_9.0,Tenure_10.0
0,619,42,0.00,1,1,1,101348.88,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,608,41,83807.86,1,0,1,112542.58,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,502,42,159660.80,3,1,0,113931.57,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,699,39,0.00,2,0,0,93826.63,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,850,43,125510.82,1,1,1,79084.10,0,0,1,...,0,1,0,0,0,0,0,0,0,0
5,645,44,113755.78,2,1,0,149756.71,1,0,1,...,0,0,0,0,0,0,0,1,0,0
6,822,50,0.00,2,1,1,10062.80,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,376,29,115046.74,4,1,0,119346.88,1,1,0,...,0,0,0,1,0,0,0,0,0,0
8,501,44,142051.07,2,0,1,74940.50,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,684,27,134603.88,1,1,1,71725.73,0,0,0,...,0,1,0,0,0,0,0,0,0,0


- Данные успешно загружены и проверены на соответствие описанию. Дубликатов нет. Типы данных преобразованы в соответствии с хранимой информацией
- Признаки RowNumber, CustomerId и Surname удалены, так как целевой признак от них никак не зависит.
- В признаке Tenure обнаружено 9% пропусков. Попытка построить классификационную модель для заполнения пропусков не принесла существенных результатов (качество модели 12%, при случайном заполнении 9%). Данный признак преобразован прямым кодированием One-Hot Encoder.

## Исследование задачи

**Подготовка**

In [16]:
train_valid, test = train_test_split(data, test_size=0.2, random_state=1234)
train, valid = train_test_split(train_valid, test_size=0.25, random_state=1234)

features_train = train.drop(['Exited'], axis=1)
target_train = train['Exited']
features_valid = valid.drop(['Exited'], axis=1)
target_valid = valid['Exited']
features_test = test.drop(['Exited'], axis=1)
target_test = test['Exited']

**Попробуем обучить различные классификационные модели без учёта дисбаланса классов**

***РЕШАЮЩЕЕ ДЕРЕВО***

Инициируем модель решающего дерева DecisionTreeClassifier<br>

Выделим гиперпараметры для настройки модели:<br>
- max_depth - максимальная глубина древа<br>
- min_samples_split - минимальное количество объектов в узле<br>
- min_samples_leaf - минимальное количество объектов в листе<br>
- criterion ("gini" или "entropy") - функция измерения качества разделения<br>

Критерием проверки качества модели будет f1 и roc_auc.

In [17]:
def decision_tree(features_train, features_valid, target_train, target_valid, cw=None):
    # в параметрах best_params_f1 и best_f1 будем хранить наилучшие параметры для модели и наивысшую f1 метрику соответственно
    best_params_f1 = []
    best_f1 = 0
    # аналогично для roc_auc метрики.
    best_params_roc_auc = []
    best_roc_auc = 0.5
    # проходим в циклах по всем выбранным гиперпараметрам
    for max_depth in range(2,10):
        for min_samples_split in range(2,6):
            for min_samples_leaf in range(1,10):
                for criterion in ['gini', 'entropy']:
                    # инициация модели с текущими гиперпараметрами
                    model_tree = DecisionTreeClassifier(random_state=123, max_depth=max_depth,
                                                        min_samples_split=min_samples_split,
                                                        min_samples_leaf=min_samples_leaf,
                                                        criterion=criterion, class_weight=cw)
                    # обучение модели на тренировочной выборке
                    model_tree.fit(features_train, target_train)
                    # поиск предсказаний модели на валидационной выбоке
                    predictions = model_tree.predict(features_valid)
                    # вычисление f1 модели методом f1_score
                    f1 = f1_score(target_valid, predictions)
                    # если текущее значение метрики f1 выше предыдущего лучшего значения, 
                    # сохраняем параметры модели и текущую метрику f1
                    if f1 > best_f1:
                        best_params_f1 = [max_depth, min_samples_split, min_samples_leaf, criterion]
                        best_f1 = f1
                    # аналогично для метрики roc_auc
                    roc_auc = roc_auc_score(target_valid, model_tree.predict_proba(features_valid)[:,1]) #КОД РЕВЬЮЕРА
                    if roc_auc > best_roc_auc:
                        best_params_roc_auc = [max_depth, min_samples_split, min_samples_leaf, criterion]
                        best_roc_auc = roc_auc
    # вывод наилучшей метрики f1 и параметров модели.
    print('''Наилучшая метрика f1: {}
    Парамтеры модели:
       max_depth = {}
       min_samples_split = {}
       min_samples_leaf = {}
       criterion = {} \n'''.format(round(best_f1, 3),
                                 best_params_f1[0],
                                 best_params_f1[1],
                                 best_params_f1[2],
                                 best_params_f1[3]
                                ))
    # вывод наилучшей метрики roc_auc и параметров модели.
    print('''Наилучшая метрика roc_auc: {}
    Парамтеры модели:
       max_depth = {}
       min_samples_split = {}
       min_samples_leaf = {}
       criterion = {} '''.format(round(best_roc_auc, 3),
                                 best_params_roc_auc[0],
                                 best_params_roc_auc[1],
                                 best_params_roc_auc[2],
                                 best_params_roc_auc[3]
                                ))
    
decision_tree(features_train, features_valid, target_train, target_valid)

Наилучшая метрика f1: 0.557
    Парамтеры модели:
       max_depth = 8
       min_samples_split = 2
       min_samples_leaf = 1
       criterion = entropy 

Наилучшая метрика roc_auc: 0.83
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 8
       criterion = gini 


- Наилучшая метрика f1 для решающего дерева 0.557 (не достаточно по техническому заданию).
- Наилучший результат по метрикам f1 и roc_auc достигается при одинаковой настройке гиперпараметров.

Попробуем улучшить результат на других моделях

***СЛУЧАЙНЫЙ ЛЕС***

Инициируем модель случайного леса RandomForestClassifierm<br>

Выделим гиперпараметры для настройки модели:<br>
- n_estimators - количество деревьев
- max_depth - максимальная глубина древа
- min_samples_split - минимальное количество объектов в узле
- min_samples_leaf - минимальное количество объектов в листе

Критерием проверки качества модели будет f1 и roc_auc.

In [18]:
def random_forest(features_train, features_valid, target_train, target_valid, cw=None):
    # в параметрах best_params_f1 и best_f1 будем хранить наилучшие параметры для модели и наивысшую f1 метрику соответственно
    best_params_f1 = []
    best_f1 = 0
    # аналогично для roc_auc метрики.
    best_params_roc_auc = []
    best_roc_auc = 0.5
    # проходим в циклах по всем выбранным гиперпараметрам
    for n_estimators in range(30,40):
        for max_depth in range(10,17,2):
            for min_samples_split in range(2,5):
                for min_samples_leaf in range(1,4):
                    # инициация модели с текущими гиперпараметрами
                    model_forest = RandomForestClassifier(random_state=123,
                                                          n_estimators=n_estimators,
                                                          max_depth=max_depth,
                                                          min_samples_split=min_samples_split,
                                                          min_samples_leaf=min_samples_leaf)
                    # обучение модели на тренировочной выборке
                    model_forest.fit(features_train, target_train)
                    # поиск предсказаний модели на валидационной выбоке
                    predictions = model_forest.predict(features_valid)

                    # вычисление f1 модели методом f1_score
                    f1 = f1_score(target_valid, predictions)
                    # если текущее значение метрики f1 выше предыдущего лучшего значения, 
                    # сохраняем параметры модели и текущую метрику f1
                    if f1 > best_f1:
                        best_params_f1 = [n_estimators, max_depth, min_samples_split, min_samples_leaf]
                        best_f1 = f1
                    # аналогично для метрики roc_auc
                    roc_auc = roc_auc = roc_auc_score(target_valid, model_forest.predict_proba(features_valid)[:,1])
                    if roc_auc > best_roc_auc:
                        best_params_roc_auc = [n_estimators, max_depth, min_samples_split, min_samples_leaf]
                        best_roc_auc = roc_auc

    # вывод наилучшей метрики f1 и параметров модели.
    print('''Наилучшая метрика f1: {}
    Парамтеры модели:
       n_estimators = {}
       max_depth = {}
       min_samples_split = {}
       min_samples_leaf = {} \n'''.format(round(best_f1, 3),
                                 best_params_f1[0],
                                 best_params_f1[1],
                                 best_params_f1[2],
                                 best_params_f1[3]
                                ))

    # вывод наилучшей метрики roc_auc и параметров модели.
    print('''Наилучшая метрика roc_auc: {}
    Парамтеры модели:
       n_estimators = {}
       max_depth = {}
       min_samples_split = {}
       min_samples_leaf = {} '''.format(round(best_roc_auc, 3),
                                        best_params_roc_auc[0],
                                        best_params_roc_auc[1],
                                        best_params_roc_auc[2],
                                        best_params_roc_auc[3]
                                       ))
random_forest(features_train, features_valid, target_train, target_valid)

Наилучшая метрика f1: 0.586
    Парамтеры модели:
       n_estimators = 30
       max_depth = 14
       min_samples_split = 2
       min_samples_leaf = 2 

Наилучшая метрика roc_auc: 0.859
    Парамтеры модели:
       n_estimators = 39
       max_depth = 16
       min_samples_split = 2
       min_samples_leaf = 2 


- Наилучшая метрика f1 для случайного леса 0.586 (не достаточно по техническому заданию).
- Наилучший результат по метрикам f1 и roc_auc достигается при одинаковой настройке гиперпараметров.

Попробуем улучшить результат на других моделях.

***ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ***

Инициируем модель случайного леса RandomForestClassifierm<br>

Выделим гиперпараметры для настройки модели:<br>
- С - инверсия силы регуляризации

Критерием проверки качества модели будет f1 и roc_auc.

In [19]:
# в параметрах best_params_f1 и best_f1 будем хранить наилучшие параметры для модели и наивысшую f1 метрику соответственно
best_params_f1 = []
best_f1 = 0
# аналогично для roc_auc метрики.
best_params_roc_auc = []
best_roc_auc = 0.5

#cs = np.linspace(0.01, 100, 1000, endpoint=True)
cs = range(1, 100)
# проходим в циклах по всем выбранным гиперпараметрам
for C in cs:
    # инициация модели с текущими гиперпараметрами
    model_regression = LogisticRegression(random_state=123,
                                      C=C,
                                      max_iter=1000)
    # обучение модели на тренировочной выборке
    model_regression.fit(features_train, target_train)
    # поиск предсказаний модели на валидационной выбоке
    predictions = model_regression.predict(features_valid)

    # вычисление f1 модели методом f1_score
    f1 = f1_score(target_valid, predictions)
    # если текущее значение метрики f1 выше предыдущего лучшего значения, 
    # сохраняем параметры модели и текущую метрику f1
    if f1 > best_f1:
        best_params_f1 = [C]
        best_f1 = f1
    # аналогично для метрики roc_auc
    roc_auc = roc_auc_score(target_valid, model_regression.predict_proba(features_valid)[:,1])
    if roc_auc > best_roc_auc:
        best_params_roc_auc = [C]
        best_roc_auc = roc_auc       

In [20]:
# вывод наилучшей метрики f1 и параметров модели.
print('''Наилучшая метрика f1: {}
Парамтеры модели:
   C = {} '''.format(round(best_f1, 3),
                             best_params_f1[0]
                            ))

# вывод наилучшей метрики roc_auc и параметров модели.
print('''Наилучшая метрика roc_auc: {}
Парамтеры модели:
   C = {} '''.format(round(best_roc_auc, 3),
                             best_params_roc_auc[0]
                            ))

Наилучшая метрика f1: 0.1
Парамтеры модели:
   C = 1 
Наилучшая метрика roc_auc: 0.658
Парамтеры модели:
   C = 1 


- Наилучшая метрика f1 для логистической регрессии 0.108 (хуже, чем на остальных моделях).
- Наилучший результат по метрикам f1 и roc_auc достигается при одинаковой настройке гиперпараметров.

Попробуем улучшить результат, разобравшись с дисбалансом классов.

**Выводы**
- Исходные данные разделены на обучающую, валидационную и тестовую выборки в соотношении 60/20/20 соответственно
- Обучены модели "Решающее Дерево", "Случайный лес", "Логистическая Регрессия" без учёта дисбаланса классов. Максимальное значение метрики f1 = 0.586 получено при обучении модели случайного леса. Качество модели не удовлетворяет условию технического задания.
- Качество моделей также оценено по метрике roc_auc. Максимальное значение метрики roc_auc = 0.859 достигнуто при обучении модели случайного леса.
- Метрики f1 и roc_auc достигли максимальных значений при одинаковых настройках гиперпараметров (но такой результат получается не всегда).

## Борьба с дисбалансом

Для борьбы с дисбалансом классов попробуем:
- взвешивание классов (class_weight='balanced')
- увеличение выборки положительного класса (копирование объектов)
- уменьшение выборки отрицательного класса (sampling)
- перемешивание объектов методом shuffle

### Взвешивание классов

In [21]:
decision_tree(features_train, features_valid, target_train, target_valid, cw='balanced')

Наилучшая метрика f1: 0.59
    Парамтеры модели:
       max_depth = 9
       min_samples_split = 2
       min_samples_leaf = 7
       criterion = gini 

Наилучшая метрика roc_auc: 0.833
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 7
       criterion = entropy 


Взвешивание классов почти не дало прироста в метрике f1 на решающем дереве.<br>
Отметим, что в данном случае наилучшие значения метрик f1 и roc_auc достигаются при различных настройках гиперпараметров.<br>
Применим upsampling и downsampling. Проверим результаты на модели случайного леса (она дала наилучший результат при начальном исследовании)

### Upsampling и downsampling

Напишем функции для увеличения и уменьшения выборок.

In [22]:
def upsample(features, target, repeat):
    """
    input:        features - признаки 
                  target - целевой признак
                  repeat (int) - множитель для повторения признаков
    output:       features_upsampled - увеличенная выборка признаков
                  target_upsampled - увеличенная выборка целевого признака
    description:  функция разделяет признаки по классам целевого признака (0, 1),
                  повторяет признаки целевого положительного класса repeat раз,
                  объединяет объекты с разными целевыми признаками методом concat,
                  перемешивает объекты методом shuffle
    """
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [23]:
def downsample(features, target, fraction):
    """
    input:        features - признаки 
                  target - целево признак
                  fraction (float) - доля объектов для сэмплирования
    output:       features_downsampled - уменьшенная выборка признаков
                  target_downsampled - уменьшенная выборка целевого признака
    description:  функция разделяет признаки по классам целевого признака (0, 1), 
                  сэмплирует признаки целевого класса 0 (с коэффициентом fraction),
                  объединяет объекты с разными целевыми признаками методом concat,
                  перемешивает объекты методом shuffle
    """
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [24]:
#Для обучающей выборки увеличим количество объектов класса 1 в 2 раза, уменьшим количество объектов класса 0 в 1.5 раза.
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 2)
features_train_balanced, target_train_balanced = downsample(features_train_upsampled, target_train_upsampled, 0.75)

*Обучим модели случайного леса и решающего дерева на сбалансированных обучающих выборках.*

In [25]:
random_forest(features_train_balanced, features_valid, target_train_balanced, target_valid)

Наилучшая метрика f1: 0.623
    Парамтеры модели:
       n_estimators = 39
       max_depth = 12
       min_samples_split = 3
       min_samples_leaf = 1 

Наилучшая метрика roc_auc: 0.857
    Парамтеры модели:
       n_estimators = 39
       max_depth = 12
       min_samples_split = 2
       min_samples_leaf = 3 


In [26]:
decision_tree(features_train_balanced, features_valid, target_train_balanced, target_valid)

Наилучшая метрика f1: 0.578
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 7
       criterion = entropy 

Наилучшая метрика roc_auc: 0.83
    Парамтеры модели:
       max_depth = 6
       min_samples_split = 2
       min_samples_leaf = 8
       criterion = entropy 


- Метрика f1 для модели "Случайный лес" достигла значения 0.623 на сбалансированной выборке, что удовлетворяет условию технического задания (>0.59).
- Для модели "Решающее дерево" метрика f1 также не достигла минимально допустимого значения 0.578.
- Наилучшие значения метрик f1 и roc_auc достигли на случайном лесе при одинаковых значениях гиперпараметров. Для решающего дерева оптимальные значения гиперпараметров различаются.
- Максимальное значение метрики roc_auc для случайного леса 0.857 не намного выше, чем для решающего дерева 0.83.

In [27]:
#Для модели случайного леса попробуем применить различные варианты upsampling и downsampling 
#(изменять параметры repeat и fraction)
for rep in range(2,4):
    for frac in np.arange(0.5, 0.8, 0.05):
        print(' ')
        print('repeat = {};   fraction = {}'.format(rep, round(frac,2)))
        features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, rep)
        features_train_balanced, target_train_balanced = downsample(features_train_upsampled, target_train_upsampled, frac)
        decision_tree(features_train_balanced, features_valid, target_train_balanced, target_valid)

 
repeat = 2;   fraction = 0.5
Наилучшая метрика f1: 0.57
    Парамтеры модели:
       max_depth = 8
       min_samples_split = 2
       min_samples_leaf = 9
       criterion = entropy 

Наилучшая метрика roc_auc: 0.832
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 9
       criterion = gini 
 
repeat = 2;   fraction = 0.55
Наилучшая метрика f1: 0.569
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 1
       criterion = gini 

Наилучшая метрика roc_auc: 0.829
    Парамтеры модели:
       max_depth = 5
       min_samples_split = 2
       min_samples_leaf = 1
       criterion = gini 
 
repeat = 2;   fraction = 0.6
Наилучшая метрика f1: 0.574
    Парамтеры модели:
       max_depth = 7
       min_samples_split = 2
       min_samples_leaf = 4
       criterion = entropy 

Наилучшая метрика roc_auc: 0.823
    Парамтеры модели:
       max_depth = 6
       min_samples_split = 2
       min_samples

Наилучший результат для решающего дерева достигнут при увеличении положительного класса тренировочной выборки в 2 раза и сэмплировании отрицательного класса с коэффициентом 0.55. При этом метрика f1 = 0.578 а roc_auc = 0.83, что не удовлетворяет условиям технического задания.

**Вывод**

Взвешивание классов с помощью параметра class_weight не дало значительного увеличения метрики f1.<br>
Upsampling положительного класса и downsampling отрицательного класса дали необходимый прирост точности моделей.<br>
Необходимое качество на валидационной выборке получили модели:
- Случайный лес:
  - f1 = 0.623
  - roc_auc = 0.857
  - repeat = 2 - повторение объектов класса 1
  - fraction = 0.75 - коэффициент сэмплирования объектов класса 0
- Решающее дерево:
  - f1 = 0.574
  - roc_auc = 0.83
  - repeat = 2
  - fraction = 0.55

## Тестирование модели

Проведём обучение полученных моделей на всех доступных данных (обучающие + валидационные), предварительно сбалансировав классы с найденными ранее коэффициентами repeat и fraction.

Вычислим метрики f1 и roc_auc на тестовой выборке.

In [28]:
#Случайный лес

features_upsampled, target_upsampled = upsample(features_train, target_train, 2)
features_balanced, target_balanced = downsample(features_upsampled, target_upsampled, 0.75)
model_forest_1 = RandomForestClassifier(random_state=123,
                                         n_estimators=33,
                                         max_depth=12,
                                         min_samples_split=3,
                                         min_samples_leaf=1)
model_forest_1.fit(features_balanced, target_balanced)
predictions = model_forest_1.predict(features_test)
f1 = f1_score(target_test, predictions)
roc_auc = roc_auc_score(target_test, model_forest_1.predict_proba(features_test)[:,1])#КОД РЕВЬЮЕРА
print(' f1 = {}, \n roc_auc = {}'.format(round(f1,2), round(roc_auc,2)))

 f1 = 0.59, 
 roc_auc = 0.85


In [30]:
#Решающее дерево

features_upsampled, target_upsampled = upsample(features_train, target_train, 2)
features_balanced, target_balanced = downsample(features_upsampled, target_upsampled, 0.75)
model_tree_1 = DecisionTreeClassifier(random_state=123,
                                         max_depth=8,
                                         min_samples_split=2,
                                         min_samples_leaf=4)
model_tree_1.fit(features_balanced, target_balanced)
predictions = model_tree_1.predict(features_test)
f1 = f1_score(target_test, predictions)
roc_auc = roc_auc_score(target_test, model_tree_1.predict_proba(features_test)[:,1]) #КОД РЕВЬЮЕРА
print(' f1 = {}, \n roc_auc = {}'.format(round(f1,2), round(roc_auc,2)))

 f1 = 0.57, 
 roc_auc = 0.82


## Вывод

Проведён анализ моделей без учёта дисбаланса классов.
- Исходные данные разделены на обучающую, валидационную и тестовую выборки.
- Обучены модели "Решающее Дерево", "Случайный лес", "Логистическая Регрессия" без учёта дисбаланса классов. Максимальное значение метрики f1 = 0.586 получено при обучении модели случайного леса.
- Качество моделей также оценено по метрике roc_auc. Максимальное значение метрики roc_auc = 0.859 достигнуто при обучении модели случайного леса.
- Метрики f1 и roc_auc достигли максимальных значений при одинаковых настройках гиперпараметров.

Проведён анализ дисбаланса классов
- Взвешивание классов с помощью параметра class_weight не дало значительного увеличения метрики f1.
- Upsampling положительного класса и downsampling отрицательного класса дали необходимый прирост точности моделей.
- Необходимое значение f1 на валидационной выборке получили модели:
 - Случайный лес:
  - f1 = 0.623
  - roc_auc = 0.857
 - Решающее дерево:
  - f1 = 0.574
  - roc_auc = 0.83
  
Проведено тестирование полученных моделей на тестовой выборке:
- Значение метрики f1 на тестовой выборке совпадают:
  - f1 = 0.59
  - roc_auc = 0.74